In [12]:
import os
if "/content" in os.getcwd():
  !rm -rf WHere_codes/
  !git clone https://github.com/keenranger/WHere_codes
  os.chdir("/content/WHere_codes")
  !pip install -r requirements.txt
from PDR.modules.DataLoader import *
from PDR.modules.Walker import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import plotly.graph_objects as go

In [14]:
file_name = "200703_taereung"
pvloader = DataLoader("data/200703_taereung.csv", file_name)
pvloader.loader()
sensor_df = pvloader.sensor_df[["time", "accx", "accy", "accz", "gyrox", "gyroy",
                                "gyroz", "rot0", "rot1", "rot2", "rot3", "game0", "game1", "game2", "game3"]]
sensor_df.head()

Data parsing... 
Data parsing done!


,time,accx,accy,accz,gyrox,gyroy,gyroz,rot0,rot1,rot2,rot3,game0,game1,game2,game3
0,1593738812495,-0.936093,3.289492,9.243615,-0.025269,-0.007327,-0.149549,0.061180,0.142934,0.842432,0.515882,0.116368,0.103181,0.542793,0.825341
1,1593738812499,-0.921728,3.361315,9.119122,-0.045427,0.033601,-0.144662,0.061548,0.142827,0.841870,0.516785,0.116253,0.103228,0.542430,0.825590
2,1593738812515,-0.986369,3.488202,8.920412,-0.143166,0.229078,-0.108621,0.061114,0.142834,0.841420,0.517568,0.115024,0.103919,0.541677,0.826169
3,1593738812535,-0.730200,3.737188,9.243615,-0.124840,0.180820,-0.109843,0.058005,0.143027,0.840678,0.519075,0.113173,0.104944,0.541310,0.826537
4,1593738812555,-0.624860,3.897593,8.743249,0.044370,-0.161875,-0.244844,0.056900,0.143152,0.839281,0.521419,0.112227,0.105646,0.540261,0.827261


In [16]:
walker = Walker()
for row in tqdm(sensor_df.itertuples(), total=len(sensor_df)):
    walker.step(row[0], row[1], row[2:5], row[5:8], row[8:12], row[12:16])


100%|██████████| 5113/5113 [00:08<00:00, 591.24it/s]


In [18]:

displacement_df = pdr_to_displacement(walker.pdr_df)
position_df = displacement_df.cumsum()
walker.pdr_df

,length,body,nav,rot,game,fusion
0,0.65,1.958852,1.915794,1.826647,1.824289,1.824289
1,0.65,1.944519,1.895287,1.809045,1.797707,1.797719
2,0.65,1.914189,1.868739,1.763508,1.751344,1.751412
3,0.65,1.925089,1.864207,1.758457,1.755877,1.756006
4,0.65,1.990057,1.861090,1.819554,1.831831,1.831972
...,...,...,...,...,...,...
200,0.65,1.052059,-0.552995,2.396656,0.124297,0.565850
201,0.65,0.766480,-0.893324,-2.547478,-0.198251,0.248164
202,0.65,0.472695,-1.241028,-1.165213,-0.522731,-0.076135
203,0.65,0.203271,-1.531803,-1.421670,-0.815428,-0.373273


In [20]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=position_df['rot_x'], 
    y=position_df['rot_y'],
    name="rot"))
fig.add_trace(go.Scatter(
    x=position_df['game_x'], 
    y=position_df['game_y'],
    name="game"))
fig.add_trace(go.Scatter(
    x=position_df['fusion_x'], 
    y=position_df['fusion_y'],
    name="fusion"))
fig.update_layout(
    title="PDR",
    width = 800,
    height = 800,
    yaxis = dict(
      scaleanchor = "x",
      scaleratio = 1,
    )
)
fig.show()

In [25]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=walker.headingcalc.heading_df['time'], y=np.rad2deg(walker.headingcalc.heading_df['rot']),name="rot"))
fig.add_trace(go.Scatter(x=walker.headingcalc.heading_df['time'], y=np.rad2deg(walker.headingcalc.heading_df['game']),name="game"))
fig.add_trace(go.Scatter(x=walker.headingcalc.heading_df['time'], y=np.rad2deg(walker.headingcalc.heading_df['fusion']),name="fusion"))
fig.update_layout(
    width = 800,
    height = 500
)
fig.show()

In [26]:
fig = go.Figure()
mag_df = pvloader.sensor_df[['time','magx', 'magy', 'magz']]
mag_norm = np.sqrt(mag_df['magx']**2 + mag_df['magy']**2 + mag_df['magz']**2)
fig.add_trace(go.Scatter(x=mag_df['time'], y=mag_norm, name="mag norm"))
fig.add_trace(go.Scatter(x=mag_df['time'], y=mag_norm, name="mag norm"))
fig.update_layout(
    width = 800,
    height = 500
)
fig.show()